In [25]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.neural_network import MLPClassifier
from sklearn import ensemble
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

### Shape Data

In [26]:
df = pd.read_csv('county_level_data.csv')

In [28]:
df = df[df.year.notnull()]

In [29]:
df = df.astype({'year':int}).astype({'year':str})

In [30]:
df.head()

,year,state_name,county,county_fips,lat,lng,population,corn_yield_per_acre,corn_price_per_bushel,land_rental_price,corn_planted_acres,total_rain,rain_days,max_temp,min_temp,too_cold_days,too_hot_days
0,2021,ILLINOIS,COOK,17031,41.8401,-87.8168,5169517,170.3,5.40,95.0,"2,860",17.279528,50.0,96.08,37.04,15.0,1.0
1,2019,ILLINOIS,COOK,17031,41.8401,-87.8168,5169517,174.0,3.55,NaN,"1,000",30.188976,71.0,96.98,42.08,18.0,2.0
2,2016,ILLINOIS,COOK,17031,41.8401,-87.8168,5169517,170.0,3.43,NaN,"3,000",24.830709,55.0,93.92,37.94,15.0,0.0
3,2021,TEXAS,BEXAR,48029,29.4489,-98.5200,1978826,73.9,6.00,43.0,"6,060",36.200787,98.0,99.50,52.34,0.0,22.0
4,2021,TEXAS,BEXAR,48029,29.4489,-98.5200,1978826,73.9,6.00,23.0,"6,060",36.200787,98.0,99.50,52.34,0.0,22.0


### Impute with Averages

In [31]:
df_grouped = df.groupby(by = ['year','state_name','county','county_fips','lat','lng','population','corn_yield_per_acre','corn_price_per_bushel','total_rain', 'rain_days', 'max_temp', 'min_temp', 'too_hot_days', 'too_cold_days']).mean().reset_index()

In [32]:
df_grouped = df_grouped.where(pd.notnull(df_grouped), None)

In [33]:
df_grouped['land_rental_price'] = df_grouped['land_rental_price'].fillna(df_grouped.groupby(['year', 'state_name'])['land_rental_price'].transform('mean'))
df_grouped['land_rental_price'] = df_grouped['land_rental_price'].fillna(df_grouped.groupby(['state_name'])['land_rental_price'].transform('mean'))

In [34]:
df_grouped = df_grouped.astype({'year':str, 'state_name':str, 'county':str, 'county_fips':str})
df_prediction = df_grouped

In [35]:
df_grouped['index'] = df_grouped.year + '-' + df_grouped.state_name + '-' + df_grouped.county + '-' + df_grouped.county_fips

In [36]:
df_grouped = df_grouped.drop(['year','state_name','county', 'county_fips'], axis=1).set_index('index')

In [37]:
df_grouped.head(5)

,lat,lng,population,corn_yield_per_acre,corn_price_per_bushel,total_rain,rain_days,max_temp,min_temp,too_hot_days,too_cold_days,land_rental_price
index,,,,,,,,,,,,
2012-ALABAMA-BLOUNT-1009,33.9809,-86.5674,57755,70.0,7.18,0.000000,0.0,100.40,50.00,4.0,0.0,36.425926
2012-ALABAMA-BULLOCK-1011,32.1005,-85.7157,10173,142.0,7.18,4.476378,23.0,98.96,46.94,5.0,2.0,12.500000
2012-ALABAMA-BUTLER-1013,31.7524,-86.6803,19726,122.2,7.18,0.000000,0.0,98.06,51.98,4.0,0.0,25.000000
2012-ALABAMA-CALHOUN-1015,33.7714,-85.8260,114324,98.7,7.18,7.311024,10.0,102.92,44.06,5.0,2.0,29.500000
2012-ALABAMA-CHEROKEE-1019,34.1759,-85.6038,26035,80.9,7.18,0.000000,0.0,100.40,37.40,5.0,7.0,44.500000


### Identify Best Model - Cross Validation on Multiple models

In [38]:
y = pd.DataFrame(df_grouped['corn_yield_per_acre'])

In [39]:
X = df_grouped.drop(['corn_yield_per_acre'], axis = 1)

In [40]:
X_scaled = pd.DataFrame(StandardScaler().fit_transform(X)).set_axis(X.columns, axis=1).set_axis(X.index, axis=0)

In [41]:
# X_scaled

In [42]:
# X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=0)

In [43]:
# CV Selection
n_samples = X.shape[0]
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)

In [44]:
xgb_params = {
    "n_estimators": 500,
    "max_depth": 4,
    "min_samples_split": 5,
    "learning_rate": 0.01,
#     "loss": "squared_error",
}

In [45]:
lasso = linear_model.Lasso()
ridge = linear_model.Ridge()
linear_regression = linear_model.LinearRegression()
rforest = RandomForestRegressor(max_depth=2, random_state=0)
svr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
xgb_model = ensemble.GradientBoostingRegressor(**xgb_params)

In [46]:
lasso_cv = cross_validate(lasso, X, y, cv=cv)
ridge_cv = cross_validate(ridge, X, y, cv=cv)
linear_cv = cross_validate(linear_regression, X, y, cv=cv)
rforest_cv = cross_validate(rforest, X, y, cv=cv)
svr_cv = cross_validate(svr, X_scaled, y, cv=cv)
xgb_cv = cross_validate(xgb_model, X_scaled, y, cv=cv)

C:\Users\camer\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\camer\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\camer\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\camer\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y wa

In [47]:
print(lasso_cv['test_score'].mean(),
ridge_cv['test_score'].mean(),
linear_cv['test_score'].mean(),
rforest_cv['test_score'].mean(),
svr_cv['test_score'].mean(),
xgb_cv['test_score'].mean()     
     )

# XGBoost is the best model

0.4062097179276316 0.4077022897092748 0.4077024542603963 0.38533539686526747 0.5058746686231316 0.6423911526759984


### Checking some baseline XGBoost Models

In [48]:
dmatrix = xgb.DMatrix(data=X, label=y)
params={'objective':'reg:squarederror'}
cv_results = xgb.cv(dtrain=dmatrix, params=params, nfold=10, metrics={'rmse'}, as_pandas=True, seed=20)
print('RMSE: %.2f' % cv_results['test-rmse-mean'].min())

RMSE: 22.46


In [49]:
dmatrix = xgb.DMatrix(data=X, label=y)
params={ 'objective':'reg:squarederror',
         'max_depth': 6, 
         'colsample_bylevel':0.5,
         'learning_rate':0.01,
         'random_state':20}
cv_results = xgb.cv(dtrain=dmatrix, params=params, nfold=10, metrics={'rmse'}, as_pandas=True, seed=20, num_boost_round=1000)
print('RMSE: %.2f' % cv_results['test-rmse-mean'].mean())

RMSE: 31.31


### GridSearchCV

In [50]:
estimator = XGBRegressor(
    objective= 'reg:squarederror',
    nthread=4,
    seed=42
)

In [51]:
parameters = {
    'max_depth': range (2, 10, 1),
    'n_estimators': range(60, 220, 40),
    'learning_rate': [0.1, 0.01, 0.05]
}

In [52]:
grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters,
    scoring = 'neg_root_mean_squared_error',
    n_jobs = 10,
    cv = 10,
    verbose=True
)

In [53]:
grid_search.fit(X, y)

Fitting 10 folds for each of 96 candidates, totalling 960 fits


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:   10.4s
[Parallel(n_jobs=10)]: Done 180 tasks      | elapsed:  1.0min
[Parallel(n_jobs=10)]: Done 430 tasks      | elapsed:  3.3min
[Parallel(n_jobs=10)]: Done 780 tasks      | elapsed:  6.5min
[Parallel(n_jobs=10)]: Done 960 out of 960 | elapsed:  8.7min finished


GridSearchCV(cv=10, error_score=nan,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    gamma=None, gpu_id=None, grow_policy=None,
                                    importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max...
                                    n_jobs=None, nthread=4,
                                    num_parallel_tree=None,
                                    objective='reg:squarederror',
                                    predictor=None, random_state=None, ...),
             iid='deprecated'

In [54]:
grid_search.best_estimator_

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.05, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=9, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=180, n_jobs=4,
             nthread=4, num_parallel_tree=1, objective='reg:squarederror',
             predictor='auto', random_state=42, ...)

In [55]:
grid_search.best_score_

-23.59324304311271

### Train Final Model

In [56]:
dmatrix = xgb.DMatrix(data=X, label=y)
params={ 'objective':'reg:squarederror',
         'max_depth': 6, 
         'colsample_bylevel':0.5,
         'learning_rate':0.05,
         'max_depth': 9,
#          'n_estimators': 180, 
         'random_state':20}
cv_results = xgb.cv(dtrain=dmatrix, params=params, nfold=10, metrics={'rmse'}, as_pandas=True, seed=20, num_boost_round=1000)
print('RMSE: %.2f' % cv_results['test-rmse-mean'].mean())

RMSE: 19.63


In [57]:
xgb_bst_model = xgb.train(dtrain=dmatrix, params=params, num_boost_round=1000)

In [59]:
xgb_bst_model.predict(dmatrix)

array([ 73.76107, 141.66962, 120.17652, ..., 183.90654, 173.29497,
       171.41188], dtype=float32)

### Develop Prediction Dataset

In [ ]:
prediction_dataset = pd.DataFrame()

In [ ]:
### Key: [min_value, max_value, interval]
ranges = {'corn_price_per_bushel': [3, 7, 2],
                     'total_rain':[0,10,2],
                     'rain_days':[0,30,10],
                     'max_temp':[90,100,10],
                     'min_temp':[0,30,10],
                     'too_hot_days':[0,6,2],
                     'too_cold_days':[4,10,2],
                     'land_rental_price':[50,100,10]}

In [ ]:
df_prediction_key = df_prediction[['state_name', 'county', 'lat', 'lng', 'population']].sort_values(['state_name', 'county']).drop_duplicates().reset_index(drop = True)

In [ ]:
rename_dict = {0:'state_name',
               1:'county',
               2:'lat',
               3:'lng',
               4:'population',
               5:'corn_price_per_bushel',
               6:'total_rain',
               7:'rain_days',
               8:'max_temp',
               9:'min_temp',
               10:'too_hot_days',
               11:'too_cold_days',
               12:'land_rental_price'}

In [ ]:
predictions_dataframe = pd.DataFrame([[0,0,0,0,0,0,0,0,0,0,0,0,0]]).rename(rename_dict,axis=1)
predictions_dataframe.head()

In [ ]:
col_names = predictions_dataframe.columns

In [ ]:
%%time
key = df_prediction_key.loc[1,:]
    
for corn_price in range(ranges['corn_price_per_bushel'][0], ranges['corn_price_per_bushel'][1], ranges['corn_price_per_bushel'][2]):
    for tot_rain in range(ranges['total_rain'][0], ranges['total_rain'][1], ranges['total_rain'][2]):
        for rain_days in range(ranges['rain_days'][0], ranges['rain_days'][1], ranges['rain_days'][2]):
            for max_temp in range(ranges['max_temp'][0], ranges['max_temp'][1], ranges['max_temp'][2]):
                for min_temp in range(ranges['min_temp'][0], ranges['min_temp'][1], ranges['min_temp'][2]):
                    for hot_days in range(ranges['too_hot_days'][0], ranges['too_hot_days'][1], ranges['too_hot_days'][2]):
                        for cold_days in range(ranges['too_cold_days'][0], ranges['too_cold_days'][1], ranges['too_cold_days'][2]):
                            for land_rental_price in range(ranges['land_rental_price'][0], ranges['land_rental_price'][1], ranges['land_rental_price'][2]):   
                                new_row = pd.Series([key['state_name'], key['county'], key['lat'], key['lng'], key['population'], corn_price, tot_rain, rain_days, max_temp, min_temp, hot_days, cold_days, land_rental_price])
                                new_row.index = col_names
                                predictions_dataframe = predictions_dataframe.append(new_row, ignore_index=True)

In [ ]:
predictions_dataframe = predictions_dataframe.drop(index=0)

In [ ]:
predictions_dataframe.head()

In [ ]:
predictions_dataframe.to_csv('prediection_dataframe.csv', index=False)

## Predicting Values on Final Dataset

In [ ]:
pred_df = pd.read_csv("prediection_dataframe.csv")

In [ ]:
dmatrix_pred_X = xgb.DMatrix(data=pred_df.iloc[:,2:])

In [ ]:
dmatrix_pred_y = bst.predict(dmatrix_pred_X)

In [ ]:
pred_df['pred_corn_yield_per_acre'] = dmatrix_pred_y

In [ ]:
pred_df.to_csv('final_results.csv', index=False)